In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



## I have learned the processes by following a course on Udemy
## The link of the course is: https://www.udemy.com/course/data-analytics-real-world-case-studies-using-python/?src=sac&kw=data+analytics+real+world
## Since I have learned everything from that course, I decided to add it as my notebook.

In [ ]:
df = pd.read_csv('/kaggle/input/jobs-on-naukricom/naukri_com-job_sample.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
count_missing = df.isnull().sum()
count_missing

In [ ]:
percent_missing = (df.isnull().sum())* (100/(len(df)))
percent_missing

#### Making a dataframe from the missing values

In [ ]:
missing_value_df = pd.DataFrame({"count_missing": count_missing,
                                 "percent_missing": percent_missing})

In [ ]:
missing_value_df.style.background_gradient(cmap='Spectral')

#### Return unique values of all the columns

In [ ]:
for col in df.columns:
    print(f"{col} has {df[col].nunique()} values")

#### A list containing row name, number of unique values and the unique values

In [ ]:
unique_df = [[col, df[col].nunique(), df[col].unique()] for col in df.columns]
unique_df[:2]

<b>We used list comprehension here. The code above works exactly like the following:
    
<code>
unique = []
for col in df.columns:
    unique.append([col, df[col].nunique(), df[col].unique()])
</code>

In [ ]:
count_df = pd.DataFrame(unique_df, columns=['col_name', 'count', 'unique'])
count_df.style.background_gradient(cmap='Spectral')

#### Cleaning the 'payrate' feature

In [ ]:
df['payrate']

Here I will find the following for each row<br>
1. Minimum Payrate<br>
2. Maximum Payrate<br>

In [ ]:
# The minimum and maximum payrate
df['payrate'][0].split(' - ')[0]

In [ ]:
min_and_max = df['payrate'][0].split(' - ')
# min_and_max =>  Gives output ['1,50,000', '2,25,000 P.A']

# We will remove ' P.A' from the second i.e. the maximum payrate, so that we get two numbers from min_and_max variable 
min_and_max[1] = min_and_max[1].split(' ')[0]   # This gives '2,25,000'

min_and_max

What we have done above only applies for those cells that have a minimum and maximum range. Other than that, the rest will have to be processed diffrently.<br>
Here we can say that, after <i>splitting</i>, if there are two values in the list, the values will surely be minimum and maximum payrate (exactly like <i><u>min_and_max</u></i>). So the length of those valid cells will be 2.<br>
We can easily find how many valid cells are there using the following method:

In [ ]:
'''
We are creating a list that will have the lengths of the lists after splitting the string around '-'
'''
len_pay = []
for pay in df['payrate']:
    len_pay.append(len(str(pay).split('-')))

In [ ]:
pd.Series(len_pay).value_counts()

So we have 4682 values that are valid entries for such processing

Here we will create a feature table that will hold all the ranges.<br>
For example, if we have a payrate like 1k-2k-3k-4k, we have to store 1k, 2k, 3k, 4k in different columns.<br>
This will be an optimal solution. 

In [ ]:
payrate_split = df['payrate'].str.split('-', expand=True)   

'''
By default the value of expand is False

The expand parameter will expand the values in different columns as following
'''

payrate_split

To properly convert the entries into integers, we have to: <br>
1. Remove extra spaces<br>
2. Remove commas

In [ ]:
payrate_split[0][0]

In [ ]:

payrate_split[0] = payrate_split[0].str.strip()

In [ ]:
payrate_split[0].str.replace(',', '')

We can use <b>Lambda</b> function to replace comma will an empty string for each entry<br>
It will do the same as the previous cell did.

In [ ]:
payrate_split[0] = payrate_split[0].apply(lambda x: str(x).replace(',', ''))

payrate_split[0]

Now we have to find whether there are any float values or not.<br>
Also we have to deal with the string values. 

Now I will get the <b>minimum pay</b> if there is float or int values<br>
We have to extract or seperate the values from the features.<br>
Possible ways:<br>
1. Exception Handling/ try except block
2. Regular expression
3. <i>to_numeric()</i> function in Pandas
4. Any function along with <i>map</i> function.

#### 1. Exception

> In the <i>try block</i>, there will be an <b>if condition</b> that will check whether the value is float. If not, then that is the missing value

<br>

> try:<br>
> &emsp; if dtype(payrate) == float<br>
> except:<br>
> &emsp; "Missing Value"

In [ ]:
pay = []      # This will contain the converted values of the first column of "payrate_split"
for payrate in payrate_split[0]:
    try:
        if type(float(payrate)) == np.float:
            pay.append(payrate)
    except:
            pay.append("missing value")


#### 2. to_numeric function

In [ ]:
pd.to_numeric(payrate_split[0], errors='coerce')

#### 3. Regular Expression

> pattern = '\D.*' <br>
> This means anything other than digits

So if I get anything other than digits, I will replace it will an empty string

In [ ]:
# A pattern that contains no numeric values
pattern = '\D.*'

In [ ]:
payrate_split[0].str.replace(pattern, '')

#### 4. "any" and "map"

In [ ]:
any(map(str.isdigit, payrate_split[0]))

In [ ]:
any(map(str.isnumeric, payrate_split[0]))

Now we will look and do something with the remaining columns of <i>payrate_split</i> <br>
The second column, or <b>payrate_split[1]</b> will have the max value. Here we have to follow some steps, which are: <br>
1. Removing all the extra white spaces.
2. Removing all the commas.
3. Removing set of characters.
4. Changing data type to float/int.


In [ ]:
payrate_split[1] = payrate_split[1].str.strip()
payrate_split[1]

In [ ]:
payrate_split[1] = payrate_split[1].apply(lambda x: str(x).replace(',', ''))

In [ ]:
pattern = '\D.*'

payrate_split[1] = payrate_split[1].str.replace(pattern, '')

In [ ]:
payrate_split.dtypes

In [ ]:
payrate_split[0] = pd.to_numeric(payrate_split[0], errors='coerce')  # For the minimum pay feature
payrate_split[1] = pd.to_numeric(payrate_split[1], errors='coerce')  # For the maximum pay feature

In [ ]:
payrate_split.dtypes

> Now we have to insert this minimum and maximum payrate in the main <i>df</i> dataframe which can be done in two ways. 

1. Defining new column in *df* for minimum and maximum payrate. (When you are adding below 5 features, just like now)
2. Concatenation Approach, using pandas library. (When you are adding more than 5-10 features)



#### Defining new columns

In [ ]:
# df['min_pay'] = payrate_split[0]
# df['max_pay'] = payrate_split[1]

### Concatenation using *pd.concat()* function

In [ ]:
pay = pd.concat([payrate_split[0], payrate_split[1]], axis=1, sort=False)
pay.columns = ['min_pay', 'max_pay']
pay

In [ ]:
df = pd.concat([df, pay], axis=1, sort=False)

In [ ]:
df.head()

### Cleaning and featurizing *'experience'*  feature

In [ ]:
df['experience'][0]

In [ ]:
# exp_list = list(df['experience'].apply(lambda x: str(x)[:-3:].strip().split(' - ')))

# exp_len = [len(i) for i in exp_list]
# # exp_list, ex_len


    def split_exp(exp):    
        min_exp = exp.split('-')[0]
        max_exp = exp.split('-')[1]
    
        return min_exp, max_exp
        
        
  We could have done the above. But we won't always have sweet data as '1 - 3 year', we may have **null** values or **1 - 3 - 5 years** type values. <br>
  So what we have to do here is as following:


In [ ]:
len1 = []

for exp in df['experience'].dropna():
    if len(exp.split('-')) != 2:
        len1.append(exp)
        
len1

#### I can drop all the rows having "Not Mentioned", which is not a handy approach. Other than this, there is also an optimal solution which is enhancing split_exp function by using "exception handling"

In [ ]:
def split_exp(exp):
    try:
        if len(exp.split('-')) == 2:
            min_exp = exp.split('-')[0]
            max_exp = exp.split('-')[1]
        return pd.Series([min_exp, max_exp])
    except:
        return pd.Series([np.nan, np.nan])
    
# To append the new data easily, we are returning pd.Series() data here. 

In [ ]:
df[['min_exp', 'max_exp']] = df['experience'].apply(split_exp).rename(columns={0:'min_exp', 1:'max_exp'})

Now we will talk about the rows where the experience value is **Not Mentioned**<br>
I will create another DataFrame for that and filter out those peculiar values.<br>
We will store those values in *nm* variable which means *not mentioned*

In [ ]:
nm = pd.DataFrame(df['experience'].str.contains("Not Mentioned"))

nm[nm['experience'] == True]

Now if I want to get the indices of these values, then I will add *.index* after the previous line of code.

In [ ]:
nm[nm['experience'] == True].index

The above values are indices of those rows with values "Not Mentioned".<br>
Now if I want to put this value into the funtion **split_exp**, I will get NaN values. 

In [ ]:
nm['experience'][1138]

In [ ]:
# The result of this code will be NaN. Because the returned values causes an exception.
split_exp(df['experience'][1138])

In [ ]:
df.head(3)

Now removing *'yrs'* from the **max_exp** column and update the column

In [ ]:
df['max_exp'] = df['max_exp'].str.replace('yrs', '')

In [ ]:
df.head(3)

In [ ]:
df.dtypes

We will convert min_exp and max_exp to float

In [ ]:
df['max_exp'] = df['max_exp'].astype(float)
df['min_exp'] = df['min_exp'].astype(float)

In [ ]:
df['avg_experience'] = (df['min_exp'] + df['max_exp'])/2
df['avg_payrate'] = (df['min_pay'] + df['max_pay'])/2

From the **min_pay, max_pay** columns and **min_exp, max_exp** columns, avg_pay and avg_exp can be derived. Which will give us more insight on the data.

### Perform Feature Engineering on **postdate** feature

##### Approaches:
1. Define own function.
2. Inbuilt functions, using datetime module
3. Optimal way: Lambda function
4. map function

#### Approach 1. Define own function

In [ ]:
df['postdate']

There are 2 types of datetime datatypes:
1. datetime64[ns]
2. <M8[ns]

both are basically same. But the entire structure depends on how numpy datatype is designed. <br>
In the following code we can see that both the datatypes are same.

In [ ]:
np.dtype('datetime64[ns]') == np.dtype('<M8[ns]')

In [ ]:
df['postdate'].dtype

Here I will write a function that will get day, month and year from each row.

In [ ]:
def fetch_dt_att(dataframe, feature):
    try:
       return pd.Series([dataframe[feature].dt.day, dataframe[feature].dt.month, dataframe[feature].dt.year])
    except:
        print('Data type is not supported')

In [ ]:
fetch_dt_att(df, 'postdate')

We have not changed the data-type of the **postdate** feature. We will do it here. 

In [ ]:
df['postdate'] = pd.to_datetime(df['postdate'])

In [ ]:
df[['day', 'month', 'year']] = fetch_dt_att(df, 'postdate')

df.head()

#### Approach 3. Using 'Lambda'

#### Approach 4. Use 'map' function

First I will define the function

In [ ]:
def fetch_dt_att2(x):
    return ([x.day, x.month, x.year])

In [ ]:
fe_date = pd.DataFrame(map(fetch_dt_att2, df['postdate'])).rename(columns={0: 'day', 1: 'month', 2: 'year'})
fe_date

We can concatenate this **fe_date** with df and will get the 3 columns. Since we have already done that using the previous steps, here I will not do it.<br>
Following is the code for it.

In [ ]:
# pd.concat([df, fe_date], axis=1)

### Prepare job_location feature

In [ ]:
df['joblocation_address'].value_counts().head(10)

Make a copy so that whatever manipulation is done, can be reverted back.

In [ ]:
data = df.copy()

In [ ]:
pd.Series(data['joblocation_address'].unique())

In [ ]:
data['joblocation_address'].value_counts().head(60).index

We can see that, there are some data repeatations. <br>
Some replacements are needed, which will be stored in dictionary. <br>
I will upload another csv file that will have the replacements.<br>
After this processing, the **duplicate subcategories** will mostly be removed. 

In [ ]:
rep = pd.read_csv(r'/kaggle/input/d/junaidmahmud/replacements/replacements.csv').set_index('Unnamed: 0')
rep

In [ ]:
replacement_dict = rep.to_dict()

In [ ]:
data.replace(replacement_dict, inplace=True, regex=True)

In [ ]:
data['joblocation_address'].value_counts()

In [ ]:
df['joblocation_address'].value_counts().head(10)

#### Now I will get some data using city names

That can be done using the following ways:
1. filter
2. Query
3. isin function

#### Approach 1. Filter

In [ ]:
loc_filter = df['joblocation_address'] == 'Noida'
loc_noida = df[loc_filter]

loc_noida.head(3)

In [ ]:
df['joblocation_address'][1760], data['joblocation_address'][1760]

So here, for the same location we get two different values and that is for the replacement of the default values in **data**.

Now we can check what are the total subcategories before and after manipulation.

In [ ]:
len(df['joblocation_address'].value_counts().index)

We can find this using the following code as well. 

In [ ]:
data['joblocation_address'].nunique()

#### Removing unnecessary columns

In [ ]:
def drop_feature(column):
    data.drop(column, axis=1, inplace=True)

In [ ]:
drop_feature(['payrate', 'experience', 'postdate', 'uniq_id'])

In [ ]:
data.columns

In [ ]:
data.head(3)

Now we will convert this data into csv

In [ ]:
data.to_csv('Job Market Analysis.csv', index=False)